This Notebook Covers:
1. Creation of Inbuilt tools
2. Creation of Custom tools
3. Creation of Agents and AgentExecutor

In [1]:
## Arxiv Research
## Tools Creation

from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper


Creation Of Inbuilt Tool

In [2]:
#Wikipedia wrapper will use the Wikipedia API to conduct searches and fetch page summaries
# By default, it will return the page summaries of the top-k results. It limits the Document content by doc_content_chars_max.
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [4]:
tools = [wiki,arxiv]

Creation of Custom Tool

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [11]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap=200).split_documents(docs)
embeddings = OllamaEmbeddings(
    model="llama3",
)
vectordb = FAISS.from_documents(documents,embeddings)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000013E74E6F4D0>, search_kwargs={})

In [12]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith-search","Search any information about langsmith")
retriever_tool.name

'langsmith-search'

In [13]:
tools = [wiki,arxiv,retriever_tool]

In [16]:
#Run all this tools with Agents and LLM Models
#Tools, LLM-->AgentExecutor

from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()
import os

groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(groq_api_key=groq_api_key,model_name = "Llama3-8b-8192")

In [18]:
#Prompt Template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

In [19]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

Creation of Agents and execution

In [20]:
#Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

In [21]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [22]:
#agent executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [23]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'What is Langsmith?'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

4. Log your first trace​
LangSmith + LangChain OSSYou don't need to use the LangSmith SDK directly if your application is built on LangChain/LangGraph (either Python and JS).See the how-to guide for tracing with LangChain here.
We provide multiple ways to log traces to LangSmith. Below, we'll highlight
how to use traceable(). See more on the Annotate code for tracing page.

View a sample output trace.
Learn more about tracing in the observability tutorials, conceptual guide and how-to guides.

Click the link printed out by your evaluation run to access the LangSmith experiments UI,
and explore the results of your evaluation.
Learn more about evaluation in the tutorials, conceptual guide, and how-to guides.
Was this page helpful?You can leave detailed feedback on GitHub.NextConcepts1. Install LangSmith2. Create an API key3. Set up yo

{'input': 'Tell me about Langsmith',
 'output': "Based on the output, it appears that Langsmith is a platform that allows users to log traces and evaluate their applications. It provides multiple ways to log traces and offers various guides and resources to help users get started.\n\nHere's a summary of what Langsmith offers:\n\n* Tracing: Langsmith allows users to log traces using the `traceable()` function, which can be used with LangChain/LangGraph applications.\n* Evaluation: Langsmith provides a way to evaluate the results of tracing, allowing users to access the results of their evaluation and explore the Langsmith experiments UI.\n* Resources: Langsmith offers various guides, tutorials, and conceptual guides to help users understand how to use the platform and get the most out of it.\n\nOverall, Langsmith seems to be a tool for developers and researchers to understand and improve the performance of their applications."}

In [24]:
agent_executor.invoke({"input":"what is difference between nlp and machine learning"})



> Entering new AgentExecutor chain...
NLP (Natural Language Processing) and Machine Learning are two interconnected fields that are often used together, but they are distinct:

NLP (Natural Language Processing) is a subfield of Artificial Intelligence (AI) that deals with the interaction between computers and humans in natural language. It focuses on enabling computers to understand, interpret, and generate human language, such as text or speech. NLP involves a range of tasks, including:

1. Tokenization: breaking text into individual words or tokens
2. Part-of-speech tagging: identifying the grammatical categories of words (e.g., noun, verb, adjective)
3. Named entity recognition: identifying specific entities (e.g., names, locations, organizations)
4. Sentiment analysis: determining the emotional tone or sentiment of text
5. Language translation: translating text from one language to another

NLP is a key component of many AI applications, such as chatbots, voice assistants, and la

{'input': 'what is difference between nlp and machine learning',
 'output': 'NLP (Natural Language Processing) and Machine Learning are two interconnected fields that are often used together, but they are distinct:\n\nNLP (Natural Language Processing) is a subfield of Artificial Intelligence (AI) that deals with the interaction between computers and humans in natural language. It focuses on enabling computers to understand, interpret, and generate human language, such as text or speech. NLP involves a range of tasks, including:\n\n1. Tokenization: breaking text into individual words or tokens\n2. Part-of-speech tagging: identifying the grammatical categories of words (e.g., noun, verb, adjective)\n3. Named entity recognition: identifying specific entities (e.g., names, locations, organizations)\n4. Sentiment analysis: determining the emotional tone or sentiment of text\n5. Language translation: translating text from one language to another\n\nNLP is a key component of many AI applicati

In [25]:
agent_executor.invoke({"input":"what's the paper 1706.03762 is all about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cBased on the output provided by the user, I can see that the tool call yielded information about the paper with ID "1706.03762". The paper is titled "Attention Is All You Need" and was published on 2023-08-02. It appears to be a research paper in the field of natural language processing and machine learning.

Here's a direct response without using any additional tools:

The paper "Attention Is All You Need" by Ashish Vaswani et al. introduces a new approach to sequence transduction that relies solely on self-attention mechanisms, eliminating the need for traditional recurrent neural network (RNN) or convolutional neural network (CNN) structures.

> F

{'input': "what's the paper 1706.03762 is all about?",
 'output': 'Based on the output provided by the user, I can see that the tool call yielded information about the paper with ID "1706.03762". The paper is titled "Attention Is All You Need" and was published on 2023-08-02. It appears to be a research paper in the field of natural language processing and machine learning.\n\nHere\'s a direct response without using any additional tools:\n\nThe paper "Attention Is All You Need" by Ashish Vaswani et al. introduces a new approach to sequence transduction that relies solely on self-attention mechanisms, eliminating the need for traditional recurrent neural network (RNN) or convolutional neural network (CNN) structures.'}